In [3]:
import numpy as np
from scipy.spatial.transform import Rotation

"""
矩阵乘法的几何意义: point或者vector的旋转和伸缩
如果规定伸缩比例为1, 矩阵就是正交阵

那么
正交阵的几何意义: point或者vector的旋转

问题:
为了产生给定的旋转效果，怎么生成这个矩阵呢？
空间中旋转需要两个参数: axis和angle

"""
def get_norm(vect) -> float:
    return sum((x**2 for x in vect))**0.5

def normalize(vect: np.ndarray, fall_back: np.ndarray = None) -> np.ndarray:
    norm = get_norm(vect)
    if norm > 0:
        return np.array(vect) / norm
    elif fall_back is not None:
        return fall_back
    else:
        return np.zeros(len(vect))
    
def rotation_matrix(angle: float, axis: np.ndarray) -> np.ndarray:
    """
    Rotation in R^3 about a specified axis of rotation.
    """
    return Rotation.from_rotvec(angle * normalize(axis)).as_matrix()


# Define the angle and axis parameters
angle = np.pi / 4 # 45 degrees in radians
axis = np.array([0, 0, 1]) # The z-axis

# Call the rotation_matrix function
rot_mat = rotation_matrix(angle, axis)

# Print the result
# 正交阵
print(rot_mat)

point = np.array([1, 1, 0])
res = np.dot(point, rot_mat.T)

print("\n")
# 忽略精度问题，绕z轴旋转，可以发现(1, 1, 0) --> (0, 1.414, 0)
# 绕着z轴逆时针旋转了45度
print(res)

[[ 0.70710678 -0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]]


[-1.11022302e-16  1.41421356e+00  0.00000000e+00]


In [9]:
import numpy as np
from scipy.spatial.transform import Rotation

"""
矩阵乘法的几何意义: point或者vector的旋转和伸缩
如果规定伸缩比例为1, 矩阵就是正交阵

那么
正交阵的几何意义: point或者vector的旋转

问题:
为了产生给定的旋转效果，怎么生成这个矩阵呢？
空间中旋转需要两个参数: axis和angle

"""
def get_norm(vect) -> float:
    return sum((x**2 for x in vect))**0.5

def normalize(vect: np.ndarray, fall_back: np.ndarray = None) -> np.ndarray:
    norm = get_norm(vect)
    if norm > 0:
        return np.array(vect) / norm
    elif fall_back is not None:
        return fall_back
    else:
        return np.zeros(len(vect))
    
def rotation_matrix(angle: float, axis: np.ndarray) -> np.ndarray:
    """
    Rotation in R^3 about a specified axis of rotation.
    """
    return Rotation.from_rotvec(angle * normalize(axis)).as_matrix()


# Define the angle and axis parameters
angle = np.pi / 4 # 45 degrees in radians
axis = np.array([0, 0, 1]) # The z-axis

# Call the rotation_matrix function
# 这里做了一个假设，axis经过原点！！！
rot_mat = rotation_matrix(angle, axis)

# Print the result
# 正交阵
print(rot_mat)

point = np.array([1, 1, 0])
"""
个人感觉, 对于point的旋转需要3个参数
axis, about_point, angle
上面的axis仅仅给出了axis的方向, 并没有给出axis经过的点

通过下面的方法
可以实现绕通过给定点的轴旋转
本质是平移
"""
about_point = np.array([1/2, 1/2, 0])
about_point = np.array([1, 1, 0])

res = np.dot(point - about_point, rot_mat.T)
res += about_point

print("\n")
print(res)

[[ 0.70710678 -0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]]


[1. 1. 0.]


In [11]:
from scipy.spatial.transform import Rotation

def quaternion_from_angle_axis(
    angle: float,
    axis: np.ndarray,
):
    return Rotation.from_rotvec(angle * normalize(axis)).as_quat()

# Define the angle and axis parameters
angle = np.pi / 4 # 45 degrees in radians
axis = np.array([0, 0, 1]) # The z-axis

# Call the rotation_matrix function
# 这里做了一个假设，axis经过原点！！！
rot_mat = rotation_matrix(angle, axis)

# Print the result
# 正交阵
print(rot_mat)

quaternion = quaternion_from_angle_axis(angle, axis) 
print(quaternion)

[[ 0.70710678 -0.70710678  0.        ]
 [ 0.70710678  0.70710678  0.        ]
 [ 0.          0.          1.        ]]
[0.         0.         0.38268343 0.92387953]


In [13]:
import math
import numpy as np
print(math.cos(np.pi/8))
print(math.sin(np.pi/8))

0.9238795325112867
0.3826834323650898


In [18]:
import numpy as np
from scipy.spatial.transform import Rotation

def angle_axis_from_quaternion(quat):
    rot_vec = Rotation.from_quat(quat).as_rotvec()
    norm = get_norm(rot_vec)
    return norm, rot_vec / norm

quat = [0, 0, 0.3826834323650898, 0.9238795325112867]
angle, axis = angle_axis_from_quaternion(quat)
print(angle, axis)

print(np.pi/4)

0.7853981633974484 [0. 0. 1.]
0.7853981633974483


In [19]:
from scipy.spatial.transform import Rotation
Rotation.identity().as_quat()

array([0., 0., 0., 1.])

In [25]:
import numpy as np
def project_along_vector(point: np.ndarray, vector: np.ndarray) -> np.ndarray:
    matrix = np.identity(3) - np.outer(vector, vector)
    return np.dot(point, matrix.T)

point = np.array([0, 2, 0])
vector = np.array([1, 1, 0])
project_along_vector(point, vector)


array([-2.,  0.,  0.])

In [32]:
from scipy.spatial.transform import Rotation

uniforms = dict()
uniforms["orientation"] = Rotation.identity().as_quat()

def get_orientation():
    '''
    获取相机旋转
    '''
    return Rotation.from_quat(uniforms["orientation"])

def get_inverse_camera_rotation_matrix():
    return get_orientation().as_matrix().T

def get_implied_camera_location() -> np.ndarray:
    '''
    获取相机的位置
    '''
    to_camera = get_inverse_camera_rotation_matrix()[2]
    dist = 16
    return dist * to_camera

def get_euler_angles():
        '''
        获取相机的欧拉角
        '''
        return get_orientation().as_euler("zxz")[::-1]

rotation = get_inverse_camera_rotation_matrix()
camera_rotation = tuple(np.array(rotation).T.flatten())
rotation
get_implied_camera_location()
get_euler_angles()

/var/folders/b0/f13r09ys4819g6vy91gl1_zr0000gn/T/ipykernel_30417/1786388265.py:27: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  return get_orientation().as_euler("zxz")[::-1]


array([0., 0., 0.])

In [1]:
def quick_sort(arr):
    # Base case: an empty or single-element array is already sorted
    if len(arr) <= 1:
        return arr

    # Choose the pivot element (can be any element in the array)
    pivot = arr[len(arr) // 2]

    # Partition the array into two sub-arrays based on the pivot
    smaller = [x for x in arr if x < pivot]
    equal = [x for x in arr if x == pivot]
    larger = [x for x in arr if x > pivot]

    # Recursive calls to sort the sub-arrays
    return quick_sort(smaller) + equal + quick_sort(larger)


In [1]:
[1,2,3] + [4,5]

[1, 2, 3, 4, 5]

In [2]:
1/0.06666666666666667

15.0